In [ ]:
#Importing Selenium
import time
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import re

## What are we scraping:
  - What fields do we need
    - Enrollment
    - Total Funding
    - Federal/State Funding
    - City Funding
  - 2015 - 2016:
  - 2016 - 2017:
    - https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2017/FY2017_Default.aspx
    - Funding Source

2016-2017 to 2017 - 2018:
https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2017/FY2017_Default.aspx
https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2017/FY2017_Default.aspx
https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2018/FY2018_Default.aspx

In [ ]:
#Set the path for the Python WebDriver
PATH = "C:\Program Files (x86)\chromedriver.exe"

#Set the path for the webdriver and website
driver = webdriver.Chrome(PATH)
driver.get("https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2017/FY2017_Default.aspx")

  - What fields do we need
    - Enrollment
    - Total Funding
    - Federal/State Funding
    - City Funding
  - 2015 - 2016:
  - 2016 - 2017:
    - https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2017/FY2017_Default.aspx
    - Funding Source

2016-2017 to 2017 - 2018:
https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2017/FY2017_Default.aspx
https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2017/FY2017_Default.aspx
https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2018/FY2018_Default.aspx

In [ ]:
#School Year 2015-2016
mainURL = "https://www.nycenet.edu/offices/d_chanc_oper/budget/exp01/y2015_2016/function.asp?R=5"
driver.get(mainURL)

#Importing the School Directory
dir = pd.read_csv('data/directory.csv')

#Initiating the output columns
district_array = []
dbn_array = []
total_funding_array = []
total_state_funding_array = []
total_federal_funding_array = []
total_enrollment_array = []
school_per_pupil_funding_array = []

#Extracting the DBN row into a List
dir_list = dir["DBN"].tolist()

for school in dir_list:
  #Searching for the school
  searchURL = 'https://www.nycenet.edu/offices/d_chanc_oper/budget/exp01/y2015_2016/function.asp?district=All&search=' + school + '&searchgo=Search&LCMS=**&GRANT=NO&cr1=All&cr2=All&cr3=All&cr4=All&R=5&prior=search'
  driver.get(searchURL)

  #Selecting the school from the dropdown menu 
  try:
    select = Select(driver.find_element_by_xpath('//*[@id="menu2"]/table/tbody/tr[2]/td/select[1]'))
    select.select_by_index(1)
    driver.find_element_by_xpath('//*[@id="menu2"]/table/tbody/tr[2]/td/input[1]').click()

    #Scraping the data
    total_school_funding_row = driver.find_elements_by_xpath("/html/body/div[1]/center[2]/table[3]/tbody/tr[position() = last()]/td")

    #Total Funding
    school_total_funding = int(re.sub('[^A-Za-z0-9]+', '', total_school_funding_row[1].text))
    total_funding_array.append(school_total_funding)

    #Total City/State Aid
    school_citystate_funding = int(re.sub('[^A-Za-z0-9]+', '', total_school_funding_row[3].text))
    total_state_funding_array.append(school_citystate_funding)

    #Total Federal State/Private Grants
    school_federal_funding = int(re.sub('[^A-Za-z0-9]+', '', total_school_funding_row[5].text))
    total_federal_funding_array.append(school_federal_funding)

    #Total Enrollment using a regex to eliminate all non-digit characters
    school_total_enrollment_element = re.findall(r'\d+', driver.find_element_by_xpath('/html/body/div[1]/center[2]/table[2]/tbody/tr/td[1]').text)
    school_total_enrollment = int(str(''.join(school_total_enrollment_element)))
    total_enrollment_array.append(school_total_enrollment)

    #School District
    school_district = int(re.findall(r'\d+', driver.find_element_by_xpath('/html/body/div[1]/center[2]/table[1]/tbody/tr[4]/td/b').text)[0])
    district_array.append(school_district)

    #School DBN:
    dbn_array.append(school)

    #Funding Per student spending:
    school_per_pupil_funding =  round(school_total_funding / school_total_enrollment, 3)

    #Rounding to 3 decimals and adding it to the array
    school_per_pupil_funding_array.append(school_per_pupil_funding)

  except Exception as e:
    print(e, "Error: "+ school +" Not Found!")

In [18]:
#School Year 2016-2017
mainURL = "https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/sber/FY2017/FY2017_Default.aspx"
driver.get(mainURL)

#Importing the School Directory
dir = pd.read_csv('data/directory.csv')

#Initiating the output columns
district_array = []
dbn_array = []
total_funding_array = []
total_state_funding_array = []
total_federal_funding_array = []
total_enrollment_array = []
school_per_pupil_funding_array = []

#Extracting the DBN row into a List
dir_list = dir["DBN"].tolist()

for school in dir_list:
  #Selecting the school from the dropdown menu 
  try:
    textbox = driver.find_element_by_id('ContentPlaceHolder1_Input_School')
    time.sleep(0.5)
    textbox.clear()
    time.sleep(0.5)
    textbox.send_keys(school)
    driver.find_element_by_id('ContentPlaceHolder1_btn_Search').click()

    #Scraping the data
    total_school_funding_row = driver.find_elements_by_xpath("/html/body/form/div[3]/div[4]/span/div[13]/table/tbody/tr[position() = last()]/td")
    #Total Funding
    school_total_funding = int(re.sub('[^A-Za-z0-9]+', '', total_school_funding_row[1].text))
    total_funding_array.append(school_total_funding)

    #Total City/State Aid
    school_citystate_funding = int(re.sub('[^A-Za-z0-9]+', '', total_school_funding_row[3].text))
    total_state_funding_array.append(school_citystate_funding)

    #Total Federal State/Private Grants
    school_federal_funding = int(re.sub('[^A-Za-z0-9]+', '', total_school_funding_row[5].text))
    total_federal_funding_array.append(school_federal_funding)

    #Total Enrollment using a regex to eliminate all non-digit characters
    school_total_enrollment_element = re.findall(r'\d+', driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_message"]/div[12]/span[1]').text)
    school_total_enrollment = int(str(''.join(school_total_enrollment_element)))
    total_enrollment_array.append(school_total_enrollment)

    #School District
    school_district = int(re.findall(r'\d+', driver.find_element_by_xpath('/html/body/form/div[3]/div[4]/span/div[11]'))[0])
    district_array.append(school_district)

    #School DBN:
    dbn_array.append(school)

    #Funding Per student spending:
    school_per_pupil_funding =  round(school_total_funding / school_total_enrollment, 3)

    #Rounding to 3 decimals and adding it to the array
    school_per_pupil_funding_array.append(school_per_pupil_funding)

    print(school_total_funding)
    print(school_citystate_funding)
    print(school_district)
    print(school_federal_funding)
    print(school_total_enrollment)

  except Exception as e:
    print(e, "Error: "+ school +" Not Found!")

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=108.0.5359.124)
 Error: M018 Not Found!
expected string or bytes-like object Error: M052 Not Found!


KeyboardInterrupt: 

In [ ]:
#Creating the Output DataFrame:
output = pd.DataFrame()

output["district"] = district_array
output["dbn"] = dbn_array
output["total enrollment"] = total_enrollment_array
output["total funding"] = total_funding_array
output["total federal funding"] = total_federal_funding_array
output["total city/state"] = total_state_funding_array
output["per pupil funding"] = school_per_pupil_funding_array

output.head()

In [ ]:
#Outputting the .csv
output.to_csv('data/2015-2016 Middle School Funding Data.csv')